# Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install islab-opendeid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00


In [ ]:
import copy
import io
import math
import os
import time
import re
import json
import random
import numpy as np
import pandas as pd
from datasets import load_dataset, Features, Value, concatenate_datasets, Dataset
from sklearn.model_selection import train_test_split
import torch
from torch.optim import lr_scheduler, AdamW
from torch.utils.data import DataLoader
from torch.nn import functional as F
from tqdm import tqdm, trange
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from transformers import get_linear_schedule_with_warmup
from islab.aicup import OpenDeidBatchSampler

# Constant

In [ ]:
"""Basic Setting"""
data_path = "/content/drive/MyDrive/AIcup/data_hpw"
model_path = "/content/drive/MyDrive/AIcup/model_hpw"

"""Data Setting"""
IgnoredPadIdx = -100
PHINullRatio = 0.3
PHINull = "PHI:Null"
PhiCategory = ['PATIENT', 'DOCTOR', 'USERNAME', 'PROFESSION',
                'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION',
                'STREET', 'CITY', 'STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER',
                'AGE', 'DATE', 'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL',
                'URL', 'IPADDR', 'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT', 'LICENSE',
                'VEHICLE', 'DEVICE', 'BIOID', 'IDNUM']

"""Model Constant Setting"""
BatchSize = 16
Epochs = int(BatchSize*0.75)
MaxLen = 196
LearningRate = 5e-5
WeightDecay = 0.02
PretrainedModel = "EleutherAI/pythia-160m"

# DataReading

In [ ]:
"""Filter PHINull"""
def filter_phi_null_ratio(dataset, ratio):
    # get indices
    phi_null_indices = [i for i, item in enumerate(dataset) if item['label'] == 'PHI:Null']
    other_indices = [i for i, item in enumerate(dataset) if item['label'] != 'PHI:Null']
    print(f"Total {len(dataset)}, PHI null {len(phi_null_indices)}, others {len(other_indices)} ")

    # target null count
    target_phi_null_count = int(min(len(phi_null_indices), len(other_indices) * ratio))
    print("target_phi_null_count", target_phi_null_count)

    if len(phi_null_indices) > target_phi_null_count:
        phi_null_indices = random.sample(phi_null_indices, target_phi_null_count)

    # reunion indices
    final_indices = phi_null_indices + other_indices
    random.shuffle(final_indices)

    final_dataset = dataset.select(final_indices)
    return final_dataset

In [ ]:
def data_reading(data_path1, data_path2):
  # read data
  data1 = load_dataset("csv", data_files=data_path1, delimiter='\t',
                       features = Features({
                          'fid': Value('string'), 'idx': Value('int64'),
                          'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)["train"]
  data2 = load_dataset("csv", data_files=data_path2, delimiter='\t',
                       features = Features({
                          'fid': Value('string'), 'idx': Value('int64'),
                          'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)["train"]

  print("pahse1 data:", len(data1))
  print("pahse2 data:", len(data2))
  print("data sample:", data1[200])

  # concatenate data
  data = concatenate_datasets([data1, data2])
  print("data length after cancatenation:", len(data))

  # filter data
  filtered_data = filter_phi_null_ratio(data, PHINullRatio)
  # print filter information
  phi_null_count_after = len([item for item in filtered_data if item['label'] == 'PHI:Null'])
  other_count_after = len(filtered_data) - phi_null_count_after
  phi_null_ratio = phi_null_count_after / other_count_after
  print("After filtering:")
  print("Length after filter:", len(filtered_data))
  print("PHI: NULL count:", phi_null_count_after)
  print("Other labels count:", other_count_after)
  print("PHI: NULL to Other labels ratio:", phi_null_ratio)
  return filtered_data

In [ ]:
# Original data paths
data_org_train_path1 = os.path.join(data_path, "train_phase1_v8_original_train.tsv")
data_org_train_path2 = os.path.join(data_path, "train_phase2_v8_original_train.tsv")
data_org_test_path1 = os.path.join(data_path, "train_phase1_v8_original_test.tsv")
data_org_test_path2 = os.path.join(data_path, "train_phase2_v8_original_test.tsv")

# Sliced data paths
data_slc_train_path1 = os.path.join(data_path, "train_phase1_v8_sliced_train.tsv")
data_slc_train_path2 = os.path.join(data_path, "train_phase2_v8_sliced_train.tsv")
data_slc_test_path1 = os.path.join(data_path, "train_phase1_v8_sliced_test.tsv")
data_slc_test_path2 = os.path.join(data_path, "train_phase2_v8_sliced_test.tsv")

# Spliced data paths
data_spl_train_path1 = os.path.join(data_path, "train_phase1_v8_spliced_train.tsv")
data_spl_train_path2 = os.path.join(data_path, "train_phase2_v8_spliced_train.tsv")
data_spl_test_path1 = os.path.join(data_path, "train_phase1_v8_spliced_test.tsv")
data_spl_test_path2 = os.path.join(data_path, "train_phase2_v8_spliced_test.tsv")

# Read datasets
data_org_train = data_reading(data_org_train_path1, data_org_train_path2)
data_org_test = data_reading(data_org_test_path1, data_org_test_path2)

data_slc_train = data_reading(data_slc_train_path1, data_slc_train_path2)
data_slc_test = data_reading(data_slc_test_path1, data_slc_test_path2)

data_spl_train = data_reading(data_spl_train_path1, data_spl_train_path2)
data_spl_test = data_reading(data_spl_test_path1, data_spl_test_path2)

In [ ]:
train_data = concatenate_datasets([data_org_train, data_slc_train, data_spl_train])
print("total train data:", len(train_data))
train_data = train_data.shuffle(seed=42)
# train_data = train_data.select(range(5000))
print("sampled train data:", len(train_data))
for i in range(100, 103):
  print("train data samples:", train_data[i])

test_data = concatenate_datasets([data_org_test, data_slc_test, data_spl_test])
test_data = test_data.shuffle(seed=42)
print("total test data:", len(test_data))
test_data = test_data.select(range(3000))
print("sampled test data:", len(test_data))
for i in range(100, 103):
  print("test data samples:", test_data[i])

# Model

In [ ]:
bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(PretrainedModel, revision="step3000")
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

<|pad|>: 50278


In [ ]:
"""model config"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = AutoConfig.from_pretrained(PretrainedModel,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False,
                                    # attention_dropout=0.7,
                                    )

model = AutoModelForCausalLM.from_pretrained(PretrainedModel, revision="step3000", config=config)
model.resize_token_embeddings(len(tokenizer))
print(model)

# frozen part of model parameters
for layer in model.gpt_neox.layers[2:10]:
    for param in layer.parameters():
        param.requires_grad = False

for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Layer frozen: {name}")

model.to(device)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

# Optimizer

In [ ]:
# optimizer = AdamW(model.parameters(), lr=LearningRate, weight_decay=WeightDecay)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LearningRate, weight_decay=WeightDecay)
num_training_steps = len(train_data) // BatchSize * Epochs
CountSteps = int(num_training_steps*0.1/Epochs) // 10 * 10
WarmUpSteps = int(num_training_steps*0.01)
print(f"training steps {num_training_steps}, count each {CountSteps} steps, warm up in {WarmUpSteps} ")

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WarmUpSteps,
    num_training_steps=num_training_steps
)

training steps 52584, count each 430 steps, warm up in 525 


# Train Dataloader

In [ ]:
def collate_batch_with_prompt_template(batch, tokenizer, template = "<|endoftext|> __CONTENT__\n\n####\n\n__LABEL__ <|END|>", IGNORED_PAD_IDX = -100):
    """ template: __CONTENT__ and __LABEL__ will be replaced with the content and the corresponding labels."""
    # default template: {bos} {data['content']} {sep}

    texts = [template.replace("__LABEL__", data['label']).replace("__CONTENT__", data['content']) for data in list(batch)]
    encoded_seq = tokenizer(texts, padding=True, truncation=True, max_length=MaxLen)

    indexed_tks = torch.tensor(encoded_seq['input_ids'])
    attention_mask = torch.tensor(encoded_seq['attention_mask'])
    encoded_label = torch.tensor(encoded_seq['input_ids'])
    encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

    return indexed_tks, encoded_label, attention_mask

In [ ]:
train_data = list(train_data)
test_data = list(test_data)

train_dataloader = DataLoader(train_data,
                              batch_sampler=OpenDeidBatchSampler(train_data, BatchSize),
                              collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                              pin_memory=True)

test_dataloader = DataLoader(test_data,
                             batch_sampler=OpenDeidBatchSampler(test_data, 1),
                             collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                             pin_memory=True)

dataloaders = {"train": train_dataloader, "test": test_dataloader}

# Training

In [ ]:
def write_losses(loss_path, train_losses, dev_losses):
  losses={}
  losses["train"] = train_losses
  losses["test"] = dev_losses
  with open(loss_path, "w") as out_config:
    json.dump(losses, out_config, indent=4)

In [ ]:
"""Train Model"""
# model path
name = str(int(time.time()))
save_path = os.path.join(model_path, name)
model_name = save_path + f"/best_{name}.pt"

loss_path = os.path.join(save_path, "loss.csv")

if not os.path.isdir(save_path):
    os.mkdir(save_path)

best_loss = float('inf')
train_losses = []
test_losses = []

for epoch in range(Epochs):
    print("[Training] Epoch {}/{}".format(epoch, Epochs - 1))
    print("-" * 10)

    running_loss_train = 0.0
    running_loss_test = 0.0
    intermediate_loss = 0.0

    # Training phase
    model.train()
    for count, (seqs, labels, masks) in enumerate(tqdm(dataloaders["train"])):
        seqs, labels, masks = seqs.to(device), labels.to(device), masks.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=seqs, labels=labels, attention_mask=masks)
        # loss = outputs.loss.mean()
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()

        running_loss_train += loss.item()
        intermediate_loss += loss.item()
        # Print training losses
        if count % CountSteps == 0 and count != 0:
            average_loss = intermediate_loss / CountSteps
            print(f"Loss in epoch{epoch}-step{count}: {average_loss:.4f}")
            train_losses.append(average_loss)
            intermediate_loss = 0

    # Print epoch train losses
    epoch_train_loss = running_loss_train / len(dataloaders["train"])
    print(f"[Training] Train Loss: {epoch_train_loss:.4f}")

    # Testing phase
    model.eval()
    with torch.no_grad():
        for seqs_test, labels_test, masks_test in tqdm(dataloaders["test"]):
            seqs_test, labels_test, masks_test = seqs_test.to(device), labels_test.to(device), masks_test.to(device)
            outputs = model(input_ids=seqs_test, labels=labels_test, attention_mask=masks_test)
            # loss = outputs.loss.mean()
            loss = outputs.loss
            running_loss_test += loss.item()

    # print epoch test loss
    epoch_test_loss = running_loss_test / len(dataloaders["test"])
    print(f"[Training] Test Loss: {epoch_test_loss:.4f}")

    # save the best model
    test_losses.append(epoch_test_loss)


    if epoch_test_loss < best_loss:
        # write losses in each epoch
        write_losses(loss_path, train_losses, test_losses)

        best_loss = epoch_test_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), model_name)
        print(f"[INFO] Updated best model on dev checkpoint: {model_name}")

[Training] Epoch 0/11
----------


  0%|          | 0/4383 [00:00<?, ?it/s]

Loss in epoch0-step430: 4.7462
Loss in epoch0-step860: 2.6183
Loss in epoch0-step1290: 2.1161
Loss in epoch0-step1720: 1.9120
Loss in epoch0-step2150: 1.7101
Loss in epoch0-step2580: 1.6178
Loss in epoch0-step3010: 1.5483
Loss in epoch0-step3440: 1.4598
Loss in epoch0-step3870: 1.4058
Loss in epoch0-step4300: 1.3767
[Training] Train Loss: 2.0372


  0%|          | 0/3000 [00:00<?, ?it/s]

[Training] Test Loss: 1.7410
[INFO] Updated best model on dev checkpoint: /content/drive/MyDrive/AIcup/model_hpw/1701425753/best_1701425753.pt
[Training] Epoch 1/11
----------


...

Loss in epoch2-step430: 1.0246
Loss in epoch2-step860: 1.0183
Loss in epoch2-step1290: 1.0160
Loss in epoch2-step1720: 1.0152
Loss in epoch2-step2150: 0.9919
Loss in epoch2-step2580: 0.9943
Loss in epoch2-step3010: 0.9988
Loss in epoch2-step3440: 0.9607
Loss in epoch2-step3870: 0.9488
Loss in epoch2-step4300: 0.9643
[Training] Train Loss: 0.9915


  0%|          | 0/3000 [00:00<?, ?it/s]

[Training] Test Loss: 1.5565
[INFO] Updated best model on dev checkpoint: /content/drive/MyDrive/AIcup/model_hpw/1701425753/best_1701425753.pt
[Training] Epoch 3/11
----------


  0%|          | 0/4383 [00:00<?, ?it/s]

Loss in epoch3-step430: 0.9036
Loss in epoch3-step860: 0.8927
Loss in epoch3-step1290: 0.8955
Loss in epoch3-step1720: 0.8968
Loss in epoch3-step2150: 0.8808
Loss in epoch3-step2580: 0.8783


# DataloaderVal

In [ ]:
def reading_validation_data(val_path):
  # read data
  val_data = load_dataset("csv", data_files=val_path, delimiter='\t',
                        features = Features({
                            'fid': Value('string'), 'idx': Value('int64'),
                            'content': Value('string'), 'label': Value('string')}),
                        column_names=['fid', 'idx', 'content', 'label'])["train"]

  print(len(val_data))
  print("data sample:", val_data[20])
  return val_data

In [ ]:
"""Get validation datasets"""
val_path_org = os.path.join(data_path, "final_original.tsv")
val_path_slc = os.path.join(data_path, "final_sliced.tsv")
val_path_spl = os.path.join(data_path, "final_spliced.tsv")

val_data_org = reading_validation_data(val_path_org)
val_data_slc = reading_validation_data(val_path_slc)
val_data_spl = reading_validation_data(val_path_spl)

# Concatenate datasets
val_data = concatenate_datasets([val_data_org, val_data_slc, val_data_spl])
data_val = list(val_data)
# val_data = list(val_data)[:100]
print("validation length:", len(val_data))
for i in range(100, 103):
  print(val_data[i])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

47042
data sample: {'fid': '1135', 'idx': 57, 'content': 'Lab No:  23F34016 ', 'label': None}


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

10625
data sample: {'fid': '1192', 'idx': 381, 'content': "n from the lump, which is in the lower part of the wound scar.  HTN. High chol. Wegener's granulomatosis. Rectal ca.", 'label': None}


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

41010
data sample: {'fid': '1135', 'idx': 99, 'content': 'Specimen: Tissue D.O.B:  3/6/1989 Sex:  F Collected: 07/08/2062 at 12:20 Location:  ENVOI Pathology-TEMORA HOSPITAL ', 'label': None}
validation length: 98677
{'fid': '1135', 'idx': 3436, 'content': 'Absence (loss) of nuclear staining for any of the mismatchrepair enzymes MLH1, PMS2, MSH2 or MSH6 is associated with microsatellite instability phenotype (MSI), and may reflect the presence of a germline mutation or somatic inactivation of that mismatch repair gene. ', 'label': None}
{'fid': '1135', 'idx': 3703, 'content': 'Preserved nuclear staining of a carcinoma for MLH1, PMS2, MSH2 and MSH6 indicates a low likelihood of microsatellite instability phenotype. ', 'label': None}
{'fid': '1135', 'idx': 3843, 'content': 'Summary - TNM 7th Edition ', 'label': None}


# Generation

In [ ]:
name = "1701423875_pythia_v5_160m_frozen"
time = name[:10]
model_name = f"drive/MyDrive/AIcup/model_hpw/{name}/best_{time}.pt"
answer_path = f"drive/MyDrive/AIcup/model_hpw/{name}/answer_{time}.txt"
prediction_path = f"drive/MyDrive/AIcup/model_hpw/{name}/prediction_{time}.txt"
model.load_state_dict(torch.load(model_name))
model = model.to(device)

In [ ]:
def aicup_predict(model, tokenizer, input, template = "<|endoftext|> __CONTENT__\n\n####\n\n"):
    seeds = [template.replace("__CONTENT__", data['content']) for data in input]
    sep = tokenizer.sep_token
    eos = tokenizer.eos_token
    pad = tokenizer.pad_token
    pad_idx = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
    """Generate text from a trained model."""
    model.eval()
    device = model.device
    texts = tokenizer(seeds, return_tensors = 'pt', padding=True, truncation=True, max_length=MaxLen).to(device)
    predictions = []

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**texts, max_new_tokens=MaxLen, pad_token_id = pad_idx,
                                        eos_token_id=tokenizer.convert_tokens_to_ids(eos))
        preds = tokenizer.batch_decode(output_tokens)
        for idx , pred in enumerate(preds):
          if "NULL" in pred:
            continue
          if sep not in pred:
            continue
          phi_infos = pred[pred.index(sep)+len(sep):].replace(pad, "").replace(eos, "").strip()
          predictions.append(f'{input[idx]["fid"]}\t{input[idx]["idx"]}\t{input[idx]["content"]}\t{phi_infos}')
    return predictions

In [ ]:
ValBatchSize = 128
with open(os.path.join(prediction_path), 'w', encoding='utf8') as f_predictions:
    for i in tqdm(range(0, len(data_val), ValBatchSize)):
        with torch.no_grad():
            seeds = data_val[i:i+ValBatchSize]
            predictions = aicup_predict(model, tokenizer, input=seeds)
            for p in predictions:
                f_predictions.write(p + '\n')

  0%|          | 0/771 [00:00<?, ?it/s]